# Best Coding Practices for Robotics 
## (based on anecdotal experience)

In this notebook, I will provide some best practices when writing code for robotic control systems. Note these ideas span beyond control and have been applied to vision, machine learning, and multi-robot coordination problems. 

# Working with Arrays and not implicit vectors (be explicit)
One lesson I learned early on was to work with arrays rather than vectors. The distinction may not be clear, but the follow examples should clear everything up for you. The takeaway message is to be as explicit as possible with linear algebra and matrix operations to avoid weird bugs that are due to careless implementation. We will be working with numpy for now and move on later with jax. For now load the library below.

In [2]:
import numpy as np

Consider the following vectors (run the cell to load them in)

In [7]:
x1 = np.array([
    [0.1],
    [0.2],
    [0.3]
])
x2 = np.array([
    0.1,0.2,0.3
])
print(
    x1,
    x1.shape, '\n',
    x2, 
    x2.shape
)

[[0.1]
 [0.2]
 [0.3]] (3, 1) 
 [0.1 0.2 0.3] (3,)


Notice that the top looks like a vector with dimension (3,1) and the array below with has dimension (3, ) . No this is not a mistake and no `x2` is not a row vector. It is an *array* which is just a special collection of numbers (list a list) that as less implicit operations than a vector. Take below the transpose and dot product. Here you can see the various ways one can code up a dot product, and accidentally introduce or remove dimensions, compute outer products, and element-wise products. This is a result of implicit functions and treating vector-vector products with dimensions instead of an explicit dot product operator that acts on two arrays that are in a vector space. 

In [30]:
res = x1.T@x1
print(res)

try: 
    res = x1@x2
except Exception as e:
    print('Throws an error')
    print(e)

print('x2 @ x2')
print('--------------------------')
res = x2@x2
print(res)
print('--------------------------')

print('x2 @ x1')
res = x2@x1
print('--------------------------')
print(res)
print('--------------------------')

print('x2.T @ x1')
# technically x2.T does not do anything to an array
res = x2.T@x1
print('--------------------------')
print(res)
print('--------------------------')

print('x1 @ x1.T')
res = x1@x1.T
print('--------------------------')
print(res)
print('--------------------------')

print('x1.T @ x2')
res = x1.T@x2
print('--------------------------')
print(res)
print('--------------------------')

print('x1 * x2')
res = x1*x2
print('--------------------------')
print(res)
print('--------------------------')

print('x1 * x1')
res = x1*x1
print('--------------------------')
print(res)
print('--------------------------')

print('x2 * x2')
res = x2*x2
print('--------------------------')
print(res)
print('--------------------------')

print('dot(x1, x2)')
res = np.dot(x2,x2)
print('--------------------------')
print(res)
print('--------------------------')


print('outer(x1, x2)')
res = np.outer(x2,x2)
print('--------------------------')
print(res)
print('--------------------------')


print('dot(A, x2), A@x2, x2@A')
A = np.array([
    [0.1,0.2,0.3],
    [0.4,0.5,0.6],
    [0.7,0.8,0.9]
])
res1 = np.dot(A,x2)
res2 = A@x2
res3 = x2@A
print('--------------------------')
print(res1,res2,res3)
print('--------------------------')

[[0.14]]
Throws an error
matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 1)
x2 @ x2
--------------------------
0.14
--------------------------
x2 @ x1
--------------------------
[0.14]
--------------------------
x2.T @ x1
--------------------------
[0.14]
--------------------------
x1 @ x1.T
--------------------------
[[0.01 0.02 0.03]
 [0.02 0.04 0.06]
 [0.03 0.06 0.09]]
--------------------------
x1.T @ x2
--------------------------
[0.14]
--------------------------
x1 * x2
--------------------------
[[0.01 0.02 0.03]
 [0.02 0.04 0.06]
 [0.03 0.06 0.09]]
--------------------------
x1 * x1
--------------------------
[[0.01]
 [0.04]
 [0.09]]
--------------------------
x2 * x2
--------------------------
[0.01 0.04 0.09]
--------------------------
dot(x1, x2)
--------------------------
0.14
--------------------------
outer(x1, x2)
--------------------------
[[0.01 0.02 0.03]
 [0.02 0.04 0.06]
 [0.03 0.

Note that in the last case, the outer product is not well defined using the @ operators with an array. An explicit outer function needs to be called. This is preferred in coding as 1) avoids ambiguous assumptions on shape/size of vectors/arrays, and 2) makes code interpretable and easy to debug. For example, imagine taking the mean of an inner product for an objective function but accidentally computing an outer product. Both operations will work, but one will be computing the correct objective function while the other will contain additional terms that skews the problem significantly. 

# Lists can be just a list of things without any special structure
Often you may just want to think of a list of things in the most abstract way possible. This is useful for example to list candidate functions for benchmarking a robot, or to try controllers that take similar inputs but may have different outputs. See below for examples: 

In [33]:
def f1(x):
    return np.mean(x)
def f2(x):
    return np.sum(np.abs(x))
def f3(x):
    return np.sum(np.log(x))
def f4(x):
    return np.mean(np.square(x))

list_of_funcs = [f1,f2,f3,f4]

# i can now loop through the functions and test various outputs! 
test_x = np.array([0.2,0.4,0.5])
i = 1
for test_f in list_of_funcs:
    print('func', i, test_f(test_x))
    i+=1

func 1 0.3666666666666667
func 2 1.1
func 3 -3.2188758248682006
func 4 0.15000000000000002


This goes beyond just functions but really any object in python can be put into lists and looped.

In [36]:
list_of_random_ararys = []
for i in range(10):
    x = np.random.normal(0., size=(3,))
    list_of_random_ararys.append(x)
print('list of arrays, NOT a matrix')
print(list_of_random_ararys)

list of arrays, NOT a matrix
[array([-0.45263161,  0.63983549,  0.89664924]), array([-0.98327863,  1.30556649,  0.00489884]), array([ 0.27178824,  0.86979423, -0.39677396]), array([ 0.09831206,  2.74529774, -1.4239546 ]), array([ 0.72851759,  0.42420449, -1.34852091]), array([-0.45400676,  1.42982614, -1.29665567]), array([0.42383257, 1.63264253, 0.23159706]), array([-0.94356872, -0.46401038,  0.18486315]), array([-1.30626874,  0.69039059, -0.80883187]), array([-0.84156873, -0.52773167, -1.19539319])]


Note that one can also take as arguments lists and convert them into other data types with properties, e.g., a matrix!

In [39]:
B = np.stack(list_of_random_ararys)
print(B.shape, '\n',B)

(10, 3) 
 [[-0.45263161  0.63983549  0.89664924]
 [-0.98327863  1.30556649  0.00489884]
 [ 0.27178824  0.86979423 -0.39677396]
 [ 0.09831206  2.74529774 -1.4239546 ]
 [ 0.72851759  0.42420449 -1.34852091]
 [-0.45400676  1.42982614 -1.29665567]
 [ 0.42383257  1.63264253  0.23159706]
 [-0.94356872 -0.46401038  0.18486315]
 [-1.30626874  0.69039059 -0.80883187]
 [-0.84156873 -0.52773167 -1.19539319]]


## Writing functions 
When writing functions, the key thing to ask yourself is what is the purpose of the function. If it has more than one purpose, you may want to split up the function. If the function can be done by a single line of code, then the function is redundant. Keep functions as self-contained as possible and be wary of global variables. If a function changes a variable in a global scope, then this has to be made explicit (try not to this by setting all global variables with a capital letter or a G in front). 

Mutable functions are also to be avoided. This is when a function takes an input argument and modifies the value of the input argument that should be held constant. This can happen by accident in python so be aware that this and double check your functions inputs/output. 

Examples TBD